# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 36 new papers today
          15 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/14 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.00814


extracting tarball to tmp_2301.00814...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.00828


extracting tarball to tmp_2301.00828...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.01093


extracting tarball to tmp_2301.01093...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.01176


extracting tarball to tmp_2301.01176...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00854-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00854) | **PHANGS-JWST First Results: Mapping the 3.3 micron Polycyclic Aromatic  Hydrocarbon Vibrational Band in Nearby Galaxies with NIRCam Medium Bands**  |
|| Karin Sandstrom, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Eva Schinnerer</mark>, <mark>Annie Hughes</mark>, <mark>Daizhong Liu</mark> |
|*Appeared on*| *2023-01-04*|
|*Comments*| *submitted to AAS journals and revised according to referee comments, part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We present maps of the 3.3 micron polycyclic aromatic hydrocarbon (PAH) emission feature in NGC 628, NGC 1365, and NGC 7496 as observed with the Near-Infrared Camera (NIRCam) imager on JWST from the PHANGS-JWST Cycle 1 Treasury project. We create maps that isolate the 3.3 micron PAH feature in the F335M filter (F335M$_{\rm PAH}$) using combinations of the F300M and F360M filters for removal of starlight continuum. This continuum removal is complicated by contamination of the F360M by PAH emission and variations in the stellar spectral energy distribution slopes between 3.0 and 3.6 micron. We modify the empirical prescription from Lai et al. (2020) to remove the starlight continuum in our highly resolved galaxies, which have a range of starlight- and PAH-dominated lines-of-sight. Analyzing radially binned profiles of the F335M$_{\rm PAH}$ emission, we find that between 5-65% of the F335M intensity comes from the 3.3 micron feature within the inner 0.5 $r_{25}$ of our targets. This percentage systematically varies from galaxy to galaxy, and shows radial trends within the galaxies related to each galaxy's distribution of stellar mass, interstellar medium, and star formation. The 3.3 micron emission is well correlated with the 11.3 micron PAH feature traced with the MIRI F1130W filter, as is expected, since both features arise from C-H vibrational modes. The average F335M$_{\rm PAH}$/F1130W ratio agrees with the predictions of recent models by Draine et al. (2021) for PAHs with size and charge distributions shifted towards larger grains with normal or higher ionization. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00881-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00881) | **PHANGS-JWST First Results: The Dust Filament Network of NGC 628 and its  Relation to Star Formation Activity**  |
|| David A. Thilker, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Francesca Pinna</mark>, <mark>Eva Schinnerer</mark>, <mark>Rowan J. Smith</mark>, <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark> |
|*Appeared on*| *2023-01-04*|
|*Comments*| *21 pages, 14 figures, accepted for publication as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| PHANGS-JWST mid-infrared (MIR) imaging of nearby spiral galaxies has revealed ubiquitous filaments of dust emission in intricate detail. We present a pilot study to systematically map the dust filament network (DFN) at multiple scales between 25-400 pc in NGC 628. MIRI images at 7.7, 10, 11.3 and 21$\mu$m of NGC 628 are used to generate maps of the filaments in emission, while PHANGS-HST B-band imaging yields maps of dust attenuation features. We quantify the correspondence between filaments traced by MIR thermal continuum / polycyclic aromatic hydrocarbon (PAH) emission and filaments detected via extinction / scattering of visible light; the fraction of MIR flux contained in the DFN; and the fraction of HII regions, young star clusters and associations within the DFN. We examine the dependence of these quantities with the physical scale at which the DFN is extracted. With our highest resolution DFN maps (25 pc filament width), we find that filaments in emission and attenuation are co-spatial in 40% of sight lines, often exhibiting detailed morphological agreement; that ~30% of the MIR flux is associated with the DFN; and that 75-80% of HII regions and 60% of star clusters younger than 5 Myr are contained within the DFN. However, the DFN at this scale is anti-correlated with looser associations of stars younger than 5 Myr identified using PHANGS-HST near-UV imaging. We discuss the impact of these findings for studies of star formation and the ISM, and the broad range of new investigations enabled with multi-scale maps of the DFN. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00828-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00828) | **From dark matter halos to pre-stellar cores: High resolution follow-up  of cosmological Lyman-Werner simulations**  |
|| <mark>Lewis R. Prole</mark>, et al. |
|*Appeared on*| *2023-01-04*|
|*Comments*| *Submitted to MNRAS ID: MN-22-5075-MJ*|
|**Abstract**| Molecular hydrogen allows cooling in primordial gas, facilitating its collapse into Population III stars within primordial halos. Lyman-Werner (LW) radiation from these stars can escape the halo and delay further star formation by destroying H$_2$ in other halos. As cosmological simulations show that increasing the background LW field strength increases the average halo mass required for star formation, we perform follow-up simulations of selected halos to investigate the knock-on effects this has on the Population III IMF. We follow 5 halos for each of the $J_{21}$ = 0, 0.01 and 0.1 LW field strengths, resolving the pre-stellar core density of $10^{-6}$ g cm$^{-3}$ (10$^{18}$ cm$^{-3}$) before inserting sink particles and following the fragmentation behaviour for hundreds of years further. We find that the mass accreted onto sinks by the end of the simulations is proportional to the mass within the $\sim 10^{-2}$ pc molecular core, which is not correlated to the initial mass of the halo. As such, the IMF shows little dependence on the LW strength. As the range of background LW field strengths tested here covers the most likely values from literature, we conclude that the IMF for so-called Pop III.2 stars is not significantly different from the initial population of Pop III.1 stars. The primordial IMF therefore likely remains unchanged until the formation of the next generation of Population II stars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01000-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01000) | **Deep Synoptic Array science: a 50 Mpc fast radio burst constrains the  mass of the Milky Way circumgalactic medium**  |
|| Vikram Ravi, et al. -- incl., <mark>Liam Connor</mark>, <mark>Greg Hellbourg</mark>, <mark>Kritti Sharma</mark> |
|*Appeared on*| *2023-01-04*|
|*Comments*| *17 pages, 8 figures, 3 tables, submitted to AAS Journals*|
|**Abstract**| We present the Deep Synoptic Array (DSA-110) discovery and interferometric localization of the so far non-repeating FRB 20220319D. The FRB originates in a young, rapidly star-forming barred spiral galaxy, IRAS 02044$+$7048, at just 50 Mpc. Although the NE2001 and YMW16 models for the Galactic interstellar-medium (ISM) contribution to the DM of FRB 20220319D exceed its total observed DM, we show that uncertainties in these models accommodate an extragalactic origin for the burst. We derive a conservative upper limit on the DM contributed by the circumgalactic medium (CGM) of the Milky Way: the limit is either 28.7 pc cm$^{-3}$ and 47.3 pc cm$^{-3}$, depending on which of two pulsars nearby on the sky to FRB 20220319D is used to estimate the ISM DM. These limits both imply that the total Galactic CGM mass is $<10^{11}M_{\odot}$, and that the baryonic mass of the Milky Way is $\lesssim60\%$ of the cosmological average given the total halo mass. More stringent albeit less conservative constraints are possible when the DMs of pulsars in the distant globular cluster M53 are additionally considered. Although our constraints are sensitive to possible anisotropy in the CGM and to the assumed form of the radial-density profile, they are not subject to uncertainties in the chemical and thermal properties of the CGM. Our results strongly support scenarios commonly predicted by galaxy-formation simulations wherein feedback processes expel baryonic matter from the halos of galaxies like the Milky Way. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01003-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01003) | **Polarised radio pulsations from a new T dwarf binary**  |
|| H. K. Vedantham, et al. -- incl., <mark>M. C. Liu</mark> |
|*Appeared on*| *2023-01-04*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Brown dwarfs display Jupiter-like auroral phenomena such as magnetospheric H$\alpha$ emission and coherent radio emission. Coherent radio emission is a probe of magnetospheric acceleration mechanisms and provides a direct measurement of the magnetic field strength at the emitter's location, both of which are difficult to access by other means. Observations of the coldest brown dwarfs (spectral types T and Y) are particularly interesting as their magnetospheric phenomena may be very similar to those in gas-giant exoplanets. Here we present 144 MHz radio and infrared adaptive optics observations of the brown dwarf WISEP J101905.63+652954.2 made using the LOFAR and Keck telescopes respectively. The radio data shows pulsed highly circularly polarised emission which yields a rotation rate of $0.32\pm0.03$ hr$^{-1}$. The infrared imaging reveals the source to be a binary with a projected separation of $423.0\pm1.6$ mas between components of spectral type T5.$5\pm0.5$ and T7.$0\pm0.5$. With a simple "toy model" we show that the radio emission can in principle be powered by the interaction between the two dwarfs with a mass-loss rates of at least $25$ times the Jovian value. WISEP J101905.63+652954.2 is interesting because it is the first pulsed methane dwarf detected in a low radio-frequency search. Unlike previous gigahertz-frequency searches that were only sensitive to objects with kiloGauss fields, our low-frequency search is sensitive to surface magnetic fields of $\approx 50$ Gauss and above which might reveal the coldest radio-loud objects down to planetary mass-scales. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01025-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01025) | **Gaussian Process Modeling Blazar Multiwavelength Variability: Indirectly  Resolving Jet Structure**  |
|| <mark>Haiyun Zhang</mark>, Dahai Yan, <mark>Li Zhang</mark> |
|*Appeared on*| *2023-01-04*|
|*Comments*| *Accepted by ApJ; 18 pages*|
|**Abstract**| Blazar jet structure can be indirectly resolved by analyzing the multiwavelength variability. In this work, we analyze the long-term variability of blazars in radio, optical and X-ray energies with the Gaussian process (GP) method. The multiwavelength variability can be successfully characterized by the damped-random walk (DRW) model. The nonthermal optical characteristic timescales of 38 blazars are statistically consistent with the $\gamma$-ray characteristic timescales of 22 blazars. For three individuals (3C 273, PKS 1510-089, and BL Lac), the nonthermal optical, X-ray, and $\gamma$-ray characteristic timescales are also consistent within the measured 95$\%$ errors, but the radio timescale of 3C 273 is too large to be constrained by the decade-long light curve. The synchrotron and inverse-Compton emissions have the same power spectral density, suggesting that the long-term jet variability is irrelevant to the emission mechanism. In the plot of the rest-frame timescale versus black hole mass, the optical-$\gamma$-ray timescales of the jet variability occupy almost the same space with the timescales of accretion disk emission from normal quasars, which may imply that the long-term variabilities of the jet and accretion disk are driven by the same physical process. It is suggested that the nonthermal optical-X-ray and $\gamma$-ray emissions are produced in the same region, while the radio core which can be resolved by very-long-baseline interferometry locates at a far more distant region from the black hole. Our study suggests a new methodology for comparing thermal and nonthermal emissions, which is achieved by using the standard GP method. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01065-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01065) | **Sub-Jovian desert of exoplanets at its boundaries: Parameter dependence  along the main sequence**  |
|| Gyula Szabó M, et al. -- incl., <mark>Viola Hegedűs</mark> |
|*Appeared on*| *2023-01-04*|
|*Comments*| *Accepted for publication in A&A; 11 pages, 11 figuress*|
|**Abstract**| Context. The lack of sub-Jovian planets on orbits of $P_{\rm orb} < 3$ days is a puzzling aspect of galaxy formation with regard to the distribution of exoplanets whose origins are currently unresolved. Aims. The possible explanations behind the formation of the sub-Jovian or Neptunian desert include several scenarios that can lead to different shapes for the boundary, predicting various dependencies between the position of the boundary and the stellar parameters. Methods. We explored the exoplanet distribution in various 2D and 3D projections, revealing the stellar-dependent substructures in the $P_{\rm orb}-M_{P}$ and the $P_{\rm orb}-R_{P}$ parameter plane. Results. We demonstrate that the upper boundary includes a range of planets, namely, inflated hot Jupiters and normal hot Jupiters, in the two parameter planes, respectively. We confirm the dependence of the boundary on several stellar parameters and, based on a fuzzy clustering analysis, we provide quantitative formulae for the dependencies in groups of smaller and larger planets. The overall period-radius distribution shows chemical substructures as well, with the boundary being dependent on volatiles and alpha-elements, alongside marginal (to none) dependence found for refractory elements. Conclusions. These findings confirm multiple plausible causes for the formation of the desert, particularly preferring those scenarios related to the irradiation-driven loss of the atmospheres of moderately massive planets as the predominant process in shaping planetary distributions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01093-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01093) | **Photochemical hazes can trace the C/O ratio in exoplanet atmospheres**  |
|| <mark>Lia Corrales</mark>, <mark>Lisseth Gavilan</mark>, D. J. Teal, Eliza M.-R. Kempton |
|*Appeared on*| *2023-01-04*|
|*Comments*| *accepted to ApJL*|
|**Abstract**| Photochemical hazes are suspected to obscure molecular features, such as water, from detection in the transmission spectra of exoplanets with atmospheric temperatures < 800 K. The opacities of laboratory produced organic compounds (tholins) from Khare et al. (1984) have become a standard for modeling haze in exoplanet atmospheres. However, these tholins were grown in an oxygen-free, Titan-like environment that is very different from typical assumptions for exoplanets, where C/O~0.5. This work presents the 0.13-10 micron complex refractive indices derived from laboratory transmission measurements of tholins grown in environments with different oxygen abundances. With the increasing uptake of oxygen, absorption increases across the entire wavelength range, and a scattering feature around 6 micron shifts towards shorter wavelengths and becomes more peaked around 5.8 micron, due to a C=O stretch resonance. Using GJ 1214b as a test-case, we examine the transmission spectra of a sub-Neptune planet with C/O ratios of solar, 1, and 1000 to evaluate the effective differences between our opacities and those of Khare. For an atmosphere with solar hydrogen and helium abundances, we find a difference of 200-1500 ppm, but for high-metallicity (Z=1000) environments, the difference may only be 20 ppm. The 1-2 micron transmission data for GJ 1214b rule out the Titan-like haze model, and are more consistent with C/O=1 and C/O=solar haze models. This work demonstrates that using haze opacities that are more consistent with underlying assumptions about bulk atmospheric composition are important for building self-consistent models that appropriately constrain the atmospheric C/O ratio, even when molecular features are obscured. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.00854.md
    + _build/html/tmp_2301.00854/./figures/fig1_panel1.png
    + _build/html/tmp_2301.00854/./figures/fig1_panel2.png
    + _build/html/tmp_2301.00854/./figures/colors_vs_f1130w_v3.png
    + _build/html/tmp_2301.00854/./figures/pah33.png
exported in  _build/html/2301.00881.md
    + _build/html/tmp_2301.00881/./FIGURES/dustletter_dustmotes.png
    + _build/html/tmp_2301.00881/./FIGURES/ngc0628_f770w_inclusionfractionYoung_filament.png
    + _build/html/tmp_2301.00881/./FIGURES/ngc0628_f2100w_inclusionfractionYoung_filament.png
    + _build/html/tmp_2301.00881/./FIGURES/ngc0628_f770w_weightedinclusionfractionYoung_filament.png
    + _build/html/tmp_2301.00881/./FIGURES/ngc0628_f2100w_weightedinclusionfractionYoung_filament.png
    + _build/html/tmp_2301.00881/./FIGURES/ngc0628_f770w_fluxfraction_filament.png
    + _build/html/tmp_2301.00881/./FIGURES/ngc0628_f2100w_fluxfraction_filament.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\obsdateone}{July 6, 2022}$
$\newcommand{\obsdatetwo}{July 15, 2022}$
$\newcommand{\mjysr}{MJy sr^{-1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\obsdateone$}{July 6, 2022}$
$\newcommand{$\obsdatetwo$}{July 15, 2022}$
$\newcommand{$\mjysr$}{MJy sr^{-1}}$</div>



<div id="title">

#  Polycyclic Aromatic Hydrocarbon Vibrational Band in Nearby Galaxies with NIRCam Medium Bands

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.00854-b31b1b.svg)](https://arxiv.org/abs/2301.00854)<mark>Appeared on: 2023-01-04</mark> - _submitted to AAS journals and revised according to referee comments, part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

Karin Sandstrom, et al. -- incl., <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>, <mark><mark>Annie Hughes</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present maps of the 3.3\micronpolycyclic aromatic hydrocarbon (PAH) emission feature in NGC 628, NGC 1365, and NGC 7496 as observed with the Near-Infrared Camera (NIRCam) imager on JWST from the PHANGS-JWST Cycle 1 Treasury project. We create maps that isolate the 3.3\micronPAH feature in the F335M filter (F335M$_{\rm PAH}$) using combinations of the F300M and F360M filters for removal of starlight continuum. This continuum removal is complicated by contamination of the F360M by PAH emission and variations in the stellar spectral energy distribution slopes between 3.0 and 3.6\micron. We modify the empirical prescription from\citet{Lai2020}to remove the starlight continuum in our highly resolved galaxies, which have a range of starlight- and PAH-dominated lines-of-sight.  Analyzing radially binned profiles of the F335M$_{\rm PAH}$emission, we find that between$5-65$\%of the F335M intensity comes from the 3.3\micronfeature within the inner 0.5$r_{25}$of our targets. This percentage systematically varies from galaxy to galaxy, and shows radial trends within the galaxies related to each galaxy's distribution of stellar mass, interstellar medium, and star formation. The 3.3\micronemission is well correlated with the 11.3\micronPAH feature traced with the MIRI F1130W filter, as is expected, since both features arise from C-H vibrational modes. The average F335M$_{\rm PAH}$/F1130W ratio agrees with the predictions of recent models by\citet{Draine2021}for PAHs with size and charge distributions shifted towards larger grains with normal or higher ionization.

</div>

<div id="div_fig1">

<img src="tmp_2301.00854/./figures/fig1_panel1.png" alt="Fig3.1" width="50%"/><img src="tmp_2301.00854/./figures/fig1_panel2.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** A representative $1.5\times 1.5$ kpc region of NGC 628 shown in the F200W, F300M, F335M, F360M, and F1130W bands with an {\em asinh} color table at each filter's native resolution. The F1130W filter traces primarily PAH emission from the 11.3 \micron feature. The F200W filter traces primarily stellar continuum.  The F335M filter is centered on the 3.3 \micron PAH feature and includes both stellar continuum and PAH emission.  Diffuse emission is visible in the F335M and F360M bands. In the F1130W panel, we highlight faint PAH emission with F1130W $<1$\mjysr with a red contour and bright PAH emission with F1130W $>10$\mjysr with a green contour.  These faint and bright selections are used in Figure \ref{fig:colorvs113}. (*fig:ngc628zoom*)

</div>
<div id="div_fig2">

<img src="tmp_2301.00854/./figures/colors_vs_f1130w_v3.png" alt="Fig4" width="100%"/>

**Figure 4. -** F335M/F300M versus F360M/F300M color in our representative region of NGC 628 selected by F1130W surface brightness. On the left, the PAH emission at F1130W is faint (F1130W $<1$\mjysr), so the 3.0-3.6 \micron colors should be dominated by stars. This region is highlighted with a red contour in Figure \ref{fig:ngc628zoom}. On the right, we select bright regions in F1130W ($>10$\mjysr), for which we expect the colors to be dominated by PAH emission. The PAH-bright region is highlighted with a green contour in Figure \ref{fig:ngc628zoom}. The red line in each panel shows the \citet{Lai2020} prescription for the stellar continuum. In starlight dominated regions, this prescription does well at predicting the continuum at F335M.  However, in regions dominated by PAHs, the F360M/F300M color is also responding to the PAH emission, leading to overestimates of the F335M continuum. We include synthetic photometry for the 1A, 1B, and 1C template spectra from \citet{Lai2020} with red symbols, illustrating that our observed colors span a much wider range than the spectra used to create the continuum subtraction prescription. (*fig:colorvs113*)

</div>
<div id="div_fig3">

<img src="tmp_2301.00854/./figures/pah33.png" alt="Fig1" width="100%"/>

**Figure 1. -** The NIRCam F300M, F335M, and F360M wavelength coverage overlaid on the 1C template spectrum from \citet{Lai2020}(square points) with their PAHFIT-based decomposition \citep{smith2007}. The continuum (including starlight, hot dust, and the effects of attenuation) is shown in blue. Orange lines show the individual dust features, including the 3.3 \micron PAH feature, the 3.4 \micron "aliphatic" emission feature and the 3.47 \micron plateau, which are labeled with arrows. The 3.74 \micron Pfund $\gamma$ emission line is shown in magenta.  The combined fit too all components is shown in green. PAH (or aliphatic) dust emission features contribute both to F335M and F360M, as well as longer wavelength dust emission in the F360M filter.  These contributions complicate continuum subtraction in our highly resolved galaxies, where individual lines of sight may be dominated by PAH emission in all three filters. (*fig:spec33*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Caltech}{\affil{California Institute of Technology, 1200 E. California Blvd., MC 249-17, Pasadena, CA 91125, USA}}$
$\newcommand{\Tamkang}{\affil{Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}}$
$\newcommand{\GEMINI}{\affil{Gemini Observatory/NSF’s NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, 85719, USA$
$}}$
$\newcommand{\ASCL}{\affil{Astrophysics Source Code Librar$
$y, Michigan Technological University, 1400 Townsend Drive, Houghton, MI 49931}}$
$\newcommand{\OSU}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{\Alberta}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\ANU}{\affil{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\IPARCOS}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\IPAC}{\affil{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\Carnegie}{\affil{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{\CCAPP}{\affil{Center for Cosmology and AstroParticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\CfA}{\affil{Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\CITEVA}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\CNRS}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ESO}{\affil{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}}$
$\newcommand{\Heidelberg}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\COOL}{\affil{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{\ICRAR}{\affil{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\IRAM}{\affil{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{\IRAP}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UPS}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ITA}{\affil{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affil{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\JHU}{\affil{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{\Leiden}{\affil{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{\Maryland}{\affil{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\MPE}{\affil{Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{\MPIA}{\affil{Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{\Nagoya}{\affil{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{\NRAO}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{\OAN}{\affil{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{\ObsParis}{\affil{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{\Princeton}{\affil{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}}$
$\newcommand{\UToledo}{\affil{University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}}$
$\newcommand{\Toulouse}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UBonn}{\affil{Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}}$
$\newcommand{\UChile}{\affil{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\UCM}{\affil{Departamento de F\'{\i}sica de la Tierra y Astrof\'{\i}sica, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\UCSD}{\affil{Center for Astrophysics and Space Sciences, Department of Physics,  University of California,\San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\ULyon}{\affil{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}}$
$\newcommand{\UMass}{\affil{University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{\UWyoming}{\affil{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\LAM}{\affil{$
$Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille),  F-13388 Marseille,$
$France}}$
$\newcommand{\UHawaii}{\affil{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{\UGent}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\IPARC}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos IPARCOS, Facultad de Ciencias F\'{\i}sicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{\STScI}{\affil{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\STScIESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\McMaster}{\affil{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\INAF}{\affil{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{\Sydney}{\affil{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{\UA}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\LERMA}{\affil{Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}}$
$\newcommand{\SAIMSU}{\affil{Sternberg Astronomical Institute, Lomonosov Moscow State University, Universitetsky pr. 13, 119234 Moscow, Russia}}$
$\newcommand{\StockholmOKC}{\affil{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\HST}{\textit{HST} }$
$\newcommand{\JWST}{\textit{JWST} }$
$\newcommand{\Gaia}{\textit{Gaia} }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\Caltech$}{\affil{California Institute of Technology, 1200 E. California Blvd., MC 249-17, Pasadena, CA 91125, USA}}$
$\newcommand{$\Tamkang$}{\affil{Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}}$
$\newcommand{\GEMINI}{\affil{Gemini Observatory/NSF’s NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, 85719, USA$
$}}$
$\newcommand{\ASCL}{\affil{Astrophysics Source Code Librar$
$y, Michigan Technological University, 1400 Townsend Drive, Houghton, MI 49931}}$
$\newcommand{$\OSU$}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{$\Alberta$}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{$\ANU$}{\affil{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{$\IPARCOS$}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{$\IPAC$}{\affil{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{$\Carnegie$}{\affil{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{$\CCAPP$}{\affil{Center for Cosmology and AstroParticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{$\CfA$}{\affil{Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{$\CITEVA$}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{$\CNRS$}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\ESO$}{\affil{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}}$
$\newcommand{$\Heidelberg$}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{$\COOL$}{\affil{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{$\ICRAR$}{\affil{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{$\IRAM$}{\affil{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{$\IRAP$}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\UPS$}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\ITA$}{\affil{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{$\IWR$}{\affil{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{$\JHU$}{\affil{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{$\Leiden$}{\affil{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{$\Maryland$}{\affil{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{$\MPE$}{\affil{Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{$\MPIA$}{\affil{Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{$\Nagoya$}{\affil{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{$\NRAO$}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{$\OAN$}{\affil{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{$\ObsParis$}{\affil{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{$\Princeton$}{\affil{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}}$
$\newcommand{$\UToledo$}{\affil{University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}}$
$\newcommand{$\Toulouse$}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\UBonn$}{\affil{Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}}$
$\newcommand{$\UChile$}{\affil{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{$\UCM$}{\affil{Departamento de F\'{\i}sica de la Tierra y Astrof\'{\i}sica, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{$\UCSD$}{\affil{Center for Astrophysics and Space Sciences, Department of Physics,  University of California,\San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{$\ULyon$}{\affil{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}}$
$\newcommand{$\UMass$}{\affil{University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{$\UWyoming$}{\affil{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\LAM}{\affil{$
$Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille),  F-13388 Marseille,$
$France}}$
$\newcommand{$\UHawaii$}{\affil{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{$\UGent$}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{$\IPARC$}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos IPARCOS, Facultad de Ciencias F\'{\i}sicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{$\STScI$}{\affil{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{$\STScI$ESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{$\McMaster$}{\affil{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{$\INAF$}{\affil{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{$\Sydney$}{\affil{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{$\UA$}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{$\LERMA$}{\affil{Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}}$
$\newcommand{$\SAIMSU$}{\affil{Sternberg Astronomical Institute, Lomonosov Moscow State University, Universitetsky pr. 13, 119234 Moscow, Russia}}$
$\newcommand{$\StockholmOKC$}{\affil{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\msun$}{M_{\odot}}$
$\newcommand{$\HST$}{\textit{HST} }$
$\newcommand{$\JWST$}{\textit{JWST} }$
$\newcommand{$\Gaia$}{\textit{Gaia} }$</div>



<div id="title">

# PHANGS-JWST First Results:  The Dust Filament Network of NGC 628 and its Relation to Star Formation Activity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.00881-b31b1b.svg)](https://arxiv.org/abs/2301.00881)<mark>Appeared on: 2023-01-04</mark> - _21 pages, 14 figures, accepted for publication as part of PHANGS-JWST ApJL Focus Issue_

</div>
<div id="authors">

David A. Thilker, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Jonathan~D.~Henshaw</mark>, <mark>Francesca~Pinna</mark>, <mark>Eva Schinnerer</mark>, <mark>Rowan J. Smith</mark>, <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark>

</div>
<div id="abstract">

**Abstract:** PHANGS-JWST mid-infrared (MIR) imaging of nearby spiral galaxies has revealed ubiquitous filaments of dust emission in intricate detail.  We present a pilot study to systematically map the dust filament network (DFN) at multiple scales between 25--400 pc in NGC 628.  MIRI images at 7.7, 10, 11.3 and 21$\micron$of NGC 628 are used to generate maps of the filaments in emission, while PHANGS-HST B-band imaging yields maps of dust attenuation features.  We quantify the correspondence between filaments traced by MIR thermal continuum / polycyclic aromatic hydrocarbon (PAH) emission and filaments detected via extinction / scattering of visible light; the fraction of MIR flux contained in the DFN; and the fraction of HII regions, young star clusters and associations within the DFN.  We examine the dependence of these quantities with the physical scale at which the DFN is extracted.  With our highest resolution DFN maps (25 pc filament width), we find that filaments in emission and attenuation are co-spatial in 40\%of sight lines, often exhibiting detailed morphological agreement; that$\sim$30\%of the MIR flux is associated with the DFN; and that 75--80\%of HII regions and 60\%of star clusters younger than 5 Myr are contained within the DFN.  However, the DFN at this scale is anti-correlated with looser associations of stars\edit1{younger than 5 Myr}identified using PHANGS-HST near-UV imaging.  We discuss the impact of these findings for studies of star formation and the ISM, and the broad range of new investigations enabled with multi-scale maps of the DFN.

</div>

<div id="div_fig1">

<img src="tmp_2301.00881/./FIGURES/dustletter_dustmotes.png" alt="Fig7" width="100%"/>

**Figure 7. -** \edit1{Dust motes and point-source MIR contamination from extremely dusty stars.}\textit{Left:} Subsection of our pre-processed B-band image, overlaid with contours corresponding to the filament masks generated at 25 pc resolution in emission, red, and absorption, blue. \textit{Center:} Unprocessed B-band image \textit{Right:}$\JWST$ F1000W image, also with the filament mask contours shown.  Yellow circles surround three example dust motes.  There are many more in the field shown, left unmarked. Green circles surround three example candidate dusty stars, others are not marked. Circles are $1$\arcsec$$ in diameter, equivalent to 48 pc at the distance of NGC 628\edit1{, and the images are oriented with North up and East left.} (*fig:dustmotes*)

</div>
<div id="div_fig2">

<img src="tmp_2301.00881/./FIGURES/ngc0628_f770w_inclusionfractionYoung_filament.png" alt="Fig8.1" width="25%"/><img src="tmp_2301.00881/./FIGURES/ngc0628_f2100w_inclusionfractionYoung_filament.png" alt="Fig8.2" width="25%"/><img src="tmp_2301.00881/./FIGURES/ngc0628_f770w_weightedinclusionfractionYoung_filament.png" alt="Fig8.3" width="25%"/><img src="tmp_2301.00881/./FIGURES/ngc0628_f2100w_weightedinclusionfractionYoung_filament.png" alt="Fig8.4" width="25%"/>

**Figure 8. -** \edit1{Testing the link of the dust filament network to current star formation.}\textit{Top:} Filament inclusion fractions for various star formation event tracers (\HII regions, young clusters, young multi-scale associations) versus the dust emission filament extraction scale. Bars indicate results for each individual extraction scale, whereas lines and points indicate the cumulative filament mask measurements. Inclusion fractions for randomly distributed points are shown with inset bars and dotted lines. \textit{Bottom:} SFR-weighted inclusion fractions.    (*fig:piegraph_InclFrac_vs_scale*)

</div>
<div id="div_fig3">

<img src="tmp_2301.00881/./FIGURES/ngc0628_f770w_fluxfraction_filament.png" alt="Fig4.1" width="50%"/><img src="tmp_2301.00881/./FIGURES/ngc0628_f2100w_fluxfraction_filament.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** \edit1{Measurements of the fraction of total flux contained within the dust filament network}\textit{Top:} Filament flux fraction in F770W expressed as percentage, with individual filament extraction scales plotted as pink bars, and cumulative multi-scale results as solid lines and points.  The faded region is shown for completeness, although we adopt a maximum cumulative scale of 200 pc. Horizontal dashes on each bar indicate the one-sigma uncertainty of the flux fraction measurements, obtained by perturbing the sky level in accord with the post-pipeline calibration described by \citet{LEROY1_PHANGSJWST}, \citet{LEE_PHANGSJWST}. Dotted lines show the one-sigma range due to uncertainty of cumulative flux fractions. \textit{Bottom:} Same, but for F2100W. (*fig:fluxfrac*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

151  publications in the last 7 days.
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers